In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [3]:
model = ResNet50(weights="imagenet")

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
img_path = "Dog.jpg"
img = image.load_img(img_path,target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

In [10]:
preds = model.predict(x)
print("Predicted :", decode_predictions(preds, top=3)[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Predicted : [('n02109525', 'Saint_Bernard', 0.9096365), ('n02108089', 'boxer', 0.038048156), ('n02109047', 'Great_Dane', 0.025297264)]


In [1]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def train(X, Y, epochs, learning_rate):
    w1 = np.random.rand(2, 2)
    b1 = np.random.rand(1, 2)
    w2 = np.random.rand(2, 1)
    b2 = np.random.rand(1, 1)

    for epoch in range(epochs):
        z1 = np.dot(X, w1) + b1
        a1 = sigmoid(z1)
        z2 = np.dot(a1, w2) + b2
        a2 = sigmoid(z2)

        error = Y - a2

        d_a2 = error * sigmoid_derivative(a2)
        d_w2 = np.dot(a1.T, d_a2)
        d_b2 = np.sum(d_a2, axis=0)

        d_a1 = np.dot(d_a2, w2.T) * sigmoid_derivative(a1)
        d_w1 = np.dot(X.T, d_a1)
        d_b1 = np.sum(d_a1, axis=0)

        w2 += learning_rate * d_w2
        b2 += learning_rate * d_b2
        w1 += learning_rate * d_w1
        b1 += learning_rate * d_b1

    return w1, b1, w2, b2

def predict(X, w1, b1, w2, b2):
    z1 = np.dot(X, w1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, w2) + b2
    a2 = sigmoid(z2)
    return np.round(a2)

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

epochs = 10000
learning_rate = 0.1
w1, b1, w2, b2 = train(X, Y, epochs, learning_rate)

for x, y in zip(X, Y):
    print(f"Input: {x}, Predicted: {predict(x, w1, b1, w2, b2)[0][0]}, Actual: {y[0]}")

Input: [0 0], Predicted: 0.0, Actual: 0
Input: [0 1], Predicted: 0.0, Actual: 1
Input: [1 0], Predicted: 1.0, Actual: 1
Input: [1 1], Predicted: 1.0, Actual: 0


In [2]:
import numpy as np

# 1. ReLU Activation Function and its Derivative
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Sigmoid activation for output layer
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 4. Mean Squared Error Loss Function
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# 6. Train function with added learning rate decay and early stopping
def train(X, Y, epochs, initial_learning_rate, decay_rate, patience):
    # 2. Architecture: Add more neurons in the hidden layer
    w1 = np.random.randn(2, 4) * np.sqrt(1 / 2)  # Xavier initialization
    b1 = np.zeros((1, 4))  # Xavier initialization

    w2 = np.random.randn(4, 1) * np.sqrt(1 / 4)  # Xavier initialization
    b2 = np.zeros((1, 1))  # Xavier initialization

    best_loss = float('inf')
    no_improvement_epochs = 0

    for epoch in range(epochs):
        # 5. Learning Rate Decay
        learning_rate = initial_learning_rate / (1 + decay_rate * epoch)

        # Forward pass
        z1 = np.dot(X, w1) + b1
        a1 = relu(z1)  # ReLU activation
        z2 = np.dot(a1, w2) + b2
        a2 = sigmoid(z2)  # Sigmoid for output

        # Compute loss (MSE)
        loss = mse_loss(Y, a2)

        # Check for early stopping condition
        if loss < best_loss:
            best_loss = loss
            no_improvement_epochs = 0
        else:
            no_improvement_epochs += 1

        if no_improvement_epochs >= patience:
            print(f"Stopping early at epoch {epoch}. Best loss: {best_loss}")
            break

        # Backpropagation
        error = Y - a2

        d_a2 = error * sigmoid_derivative(a2)
        d_w2 = np.dot(a1.T, d_a2)
        d_b2 = np.sum(d_a2, axis=0)

        d_a1 = np.dot(d_a2, w2.T) * relu_derivative(a1)  # ReLU derivative in hidden layer
        d_w1 = np.dot(X.T, d_a1)
        d_b1 = np.sum(d_a1, axis=0)

        # Update weights and biases with learning rate decay
        w2 += learning_rate * d_w2
        b2 += learning_rate * d_b2
        w1 += learning_rate * d_w1
        b1 += learning_rate * d_b1

    return w1, b1, w2, b2

# Predict function
def predict(X, w1, b1, w2, b2):
    z1 = np.dot(X, w1) + b1
    a1 = relu(z1)  # ReLU activation in hidden layer
    z2 = np.dot(a1, w2) + b2
    a2 = sigmoid(z2)  # Sigmoid activation in output layer
    return np.round(a2)

# Dataset (XOR logic gate)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

# Hyperparameters
epochs = 10000
initial_learning_rate = 0.1  # Starting learning rate
decay_rate = 0.001  # Learning rate decay factor
patience = 100  # Early stopping patience

# Train the model
w1, b1, w2, b2 = train(X, Y, epochs, initial_learning_rate, decay_rate, patience)

# Test predictions
for x, y in zip(X, Y):
    print(f"Input: {x}, Predicted: {predict(x, w1, b1, w2, b2)[0][0]}, Actual: {y[0]}")


Input: [0 0], Predicted: 0.0, Actual: 0
Input: [0 1], Predicted: 1.0, Actual: 1
Input: [1 0], Predicted: 1.0, Actual: 1
Input: [1 1], Predicted: 0.0, Actual: 0
